# Re-create the BBB data

A dataset like BBB doesn't exist in companies in its raw form
Someone has to create it first ... likely from different data sources!

The goal of this assignment is to re-create the pandas data frame in bbb.pkl
data EXACTLY from its components. Follow the steps outlined below:

1. Determine how to load the different file types (use pd.read_pickel, pd.read_csv,
   pd.read_excel, and sqlite3.connect)
2. Determine what data transformations are needed and how the data should be
   combined into a data frame. You MUST name your re-created data frame 'bbb_rec'
3. Your work should be completely reproducible (i.e., generate the same results on
   another computer). Think about the 'paths' you are using to load the data. Will
   I or the TA have access to those same directories? Of course you cannot 'copy'
   any data from bbb into bbb_rec. You can copy the data description however
4. The final step will be to check that your code produces a data frame
   identical to the pandas data frame in the bbb.pkl file, using pandas' "equals"
   method shown below. If the test passes, write bbb_rec to "data/bbb_rec.pkl". Do
   NOT change the test as this will be used in grading/evaluation
5. Make sure to style your python code appropriately for easy readable
6. When you are done, save your, code and commit and push your work to GitLab.
   Of course you can commit and push as often as you like, but only before the
   due date. Late assignments will not be accepted
7. When testing your (final) code make sure to restart the kernel regularly.
   Restarting the kernel ensures that all modules and variables your code needs
   are actually generated and loaded in your code
8. You can use modules other than the ones mentioned below but do NOT use
   modules that are not part of the rsm-msba-spark docker container by default

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import date
import pyrsm as rsm
import urllib.request
from tempfile import NamedTemporaryFile as tmpfile
import os
import math

load the original bbb.pkl data frame from a Dropbox link

In [ ]:
bbb_file = tmpfile().name
urllib.request.urlretrieve(
    "https://www.dropbox.com/s/6bulog0ij4pr52o/bbb.pkl?dl=1", bbb_file
)
bbb = pd.read_pickle(bbb_file)

In [ ]:
bbb.head()

,acctnum,gender,state,zip,zip3,first,last,book,nonbook,total,purch,child,youth,cook,do_it,reference,art,geog,buyer,training
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2,no,1
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1,no,1
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0,no,0
3,10004,F,NJ,07016,070,7,7,15,257,272,1,0,0,0,0,1,0,0,no,0
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0,no,1


view the data description of the original data to determine
what needs to be re-created

In [ ]:
rsm.describe(bbb)

# BookBinders Book Club

Information on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below

## Variables

* acctnum: Customer account number
* gender: Customer gender - M=male, F=female
* state: State where customer lives (2-character abbreviation)
* zip: ZIP code (5-digit)
* zip3: First 3 digits of ZIP code
* first: Number of months since first purchase
* last: Number of months since most recent purchase
* book: Total dollars spent on books
* nonbook: Total dollars spent on non-book products
* total: Total dollars spent
* purch: Total number of books purchased
* child: Total number of children's books purchased
* youth: Total number of youth books purchased
* cook: Total number of cook books purchased 
* do_it: Total number of do-it-yourself books purchased 
* reference: Total number of reference books purchased 
* art: Total number of art books purchased 
* geog: Total number of geography books purchased 
* buyer: Did the customer buy The Art History of Florence?  (yes, no)
* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset

Check that the working directory you are using is the same as the location of this file

In [ ]:
os.getcwd()

'/home/jovyan/git/rsm-mgta455-bbb-recreate-data'

load demographics data from bbb_demographics.tsv

In [ ]:
Dem_data = pd.read_csv("data/bbb_demographics.tsv", sep = "\t")

In [ ]:
nonbook_data = pd.read_excel("data/bbb_nonbook.xls")

In [ ]:
nonbook_data.head()

,acctnum,nonbook
0,10001,248
1,10002,103
2,10003,147
3,10004,257
4,10005,134


load nonbook aggregate spending from bbb_nonbook.xls

In [ ]:
Dem_data.head()

,acctnum,gender,state,zip
0,10001,M,NY,10605
1,10002,M,NY,10960
2,10003,F,PA,19146
3,10004,F,NJ,7016
4,10005,F,NY,10804


In [ ]:
Dem_data.dtypes

acctnum     int64
gender     object
state      object
zip         int64
dtype: object

load purchase and buy-no-buy information from bbb.sqlite
hint: what data type is "date" in the database?
hint: most systems record dates internally as the number
of days since some origin. You can use the pd.to_datetime
method to convert the number to a date with argument: origin = "1-1-1970"

In [ ]:
con = sqlite3.connect("data/bbb.sqlite")

In [ ]:
def db_list_tables(con):
    """Return all table names"""
    cursor = con.cursor()
    cursor.execute("select name from sqlite_master where type='table';")
    return [x[0] for x in cursor.fetchall()]


def db_list_fields(con, tabel):
    """Return all column names for a specified table"""
    cursor = con.cursor()
    cursor.execute(f"select * from {tabel} limit 1;")
    return [name[0] for name in cursor.description]

In [ ]:
db_list_tables(con)

['buyer', 'purchase']

In [ ]:
db_list_fields(con, 'buyer')

['acctnum', 'buyer', 'training']

In [ ]:
db_list_fields(con, 'purchase')

['acctnum', 'date', 'purchase', 'price']

In [ ]:
buyer_data = pd.read_sql_query("SELECT * from buyer", con)

In [ ]:
buyer_data.head()

,acctnum,buyer,training
0,10001,no,1
1,10002,no,1
2,10003,no,0
3,10004,no,0
4,10005,no,1


In [ ]:
buyer_data['acctnum'] = buyer_data['acctnum'].astype(int)

In [ ]:
buyer_data.dtypes

acctnum      int64
buyer       object
training     int64
dtype: object

In [ ]:
purchase_data = pd.read_sql_query("SELECT * from purchase", con)

In [ ]:
purchase_data.head()

,acctnum,date,purchase,price
0,10001,13187.0,geog,11.0
1,10001,13191.0,cook,11.0
2,10001,13266.0,youth,9.0
3,10001,13371.0,youth,9.0
4,10001,13457.0,child,10.0


In [ ]:
purchase_data.dtypes

acctnum      object
date        float64
purchase     object
price       float64
dtype: object

In [ ]:
purchase_data['date'] = pd.to_datetime(purchase_data['date'],unit='d',origin="1-1-1970")

In [ ]:
purchase_data.head()

,acctnum,date,purchase,price
0,10001,2006-02-08,geog,11.0
1,10001,2006-02-12,cook,11.0
2,10001,2006-04-28,youth,9.0
3,10001,2006-08-11,youth,9.0
4,10001,2006-11-05,child,10.0


In [ ]:
purchase_data.dtypes

acctnum             object
date        datetime64[ns]
purchase            object
price              float64
dtype: object

In [ ]:
## For Dem_data we first add preceeding 0s to make the overall zip of 5 digit
Dem_data['zip']=Dem_data['zip'].apply(lambda x: '{0:0>5}'.format(x))
Dem_data.head()

,acctnum,gender,state,zip
0,10001,M,NY,10605
1,10002,M,NY,10960
2,10003,F,PA,19146
3,10004,F,NJ,07016
4,10005,F,NY,10804


In [ ]:
Dem_data['zip_str'] = Dem_data['zip'].astype(str) 
Dem_data.head()

,acctnum,gender,state,zip,zip_str
0,10001,M,NY,10605,10605
1,10002,M,NY,10960,10960
2,10003,F,PA,19146,19146
3,10004,F,NJ,07016,07016
4,10005,F,NY,10804,10804


In [ ]:
Dem_data['zip3'] = Dem_data['zip_str'].str[0:3]
Dem_data.head()

,acctnum,gender,state,zip,zip_str,zip3
0,10001,M,NY,10605,10605,106
1,10002,M,NY,10960,10960,109
2,10003,F,PA,19146,19146,191
3,10004,F,NJ,07016,07016,070
4,10005,F,NY,10804,10804,108


add the zip3 variable

In [ ]:
Dem_data = Dem_data.drop(['zip_str'], axis = 1)
Dem_data.head()

,acctnum,gender,state,zip,zip3
0,10001,M,NY,10605,106
1,10002,M,NY,10960,109
2,10003,F,PA,19146,191
3,10004,F,NJ,07016,070
4,10005,F,NY,10804,108


In [ ]:
##Dem_data

use the following reference date (i.e., "today" for the analysis)

In [ ]:
start_date = date(2010, 3, 8)


def diff_months(date1, date2):
    """
    This function calculates the difference in months between
    date1 and date2 when a customer purchased a product
    """
    y = date1.year - date2.year
    m = date1.month - date2.month
    return y * 12 + m

In [ ]:
diff_months(date.today(),start_date)

141

In [ ]:
purchase_data

,acctnum,date,purchase,price
0,10001,2006-02-08,geog,11.0
1,10001,2006-02-12,cook,11.0
2,10001,2006-04-28,youth,9.0
3,10001,2006-08-11,youth,9.0
4,10001,2006-11-05,child,10.0
...,...,...,...,...
194506,60000,2007-10-08,geog,9.0
194507,60000,2007-10-25,geog,14.0
194508,60000,2009-03-27,art,10.0
194509,60000,2010-01-07,child,14.0


In [ ]:
## Sorting by date and grouping by accountnum to get the first and last data

In [ ]:
try2 = purchase_data.groupby(['acctnum'])

In [ ]:
try3 = try2.agg(Minimum_Date=('date', np.min), Maximum_Date=('date', np.max))

In [ ]:
try3

,Minimum_Date,Maximum_Date
acctnum,,
10001,2006-02-08,2007-10-08
10002,2006-12-08,2007-12-08
10003,2008-08-08,2008-12-08
10004,2009-08-08,2009-08-08
10005,2008-12-08,2008-12-08
...,...,...
59996,2009-06-08,2009-06-08
59997,2008-02-08,2009-10-08
59998,2009-12-08,2009-12-08


In [ ]:
## Trying out to get the "first" by today - oldest date

In [ ]:
try3['first'] = ((pd.to_datetime(start_date) - try3['Minimum_Date']) / np.timedelta64(1, 'M')).round()

In [ ]:
try3['last'] = ((pd.to_datetime(start_date) - try3['Maximum_Date']) / np.timedelta64(1, 'M')).round()

In [ ]:
try3['first'] = try3['first'].astype(int)
try3['last'] = try3['last'].astype(int)

In [ ]:
### From here we get the first and last
try3

,Minimum_Date,Maximum_Date,first,last
acctnum,,,,
10001,2006-02-08,2007-10-08,49,29
10002,2006-12-08,2007-12-08,39,27
10003,2008-08-08,2008-12-08,19,15
10004,2009-08-08,2009-08-08,7,7
10005,2008-12-08,2008-12-08,15,15
...,...,...,...,...
59996,2009-06-08,2009-06-08,9,9
59997,2008-02-08,2009-10-08,25,5
59998,2009-12-08,2009-12-08,3,3


In [ ]:
try3 = try3.drop(['Minimum_Date','Maximum_Date'], axis = 1)
try3

,first,last
acctnum,,
10001,49,29
10002,39,27
10003,19,15
10004,7,7
10005,15,15
...,...,...
59996,9,9
59997,25,5
59998,3,3


In [ ]:
try3 = try3.reset_index()
try3['acctnum'] = try3['acctnum'].astype(int)

In [ ]:
try3

,acctnum,first,last
0,10001,49,29
1,10002,39,27
2,10003,19,15
3,10004,7,7
4,10005,15,15
...,...,...,...
49995,59996,9,9
49996,59997,25,5
49997,59998,3,3
49998,59999,49,29


In [ ]:
purchase_data

,acctnum,date,purchase,price
0,10001,2006-02-08,geog,11.0
1,10001,2006-02-12,cook,11.0
2,10001,2006-04-28,youth,9.0
3,10001,2006-08-11,youth,9.0
4,10001,2006-11-05,child,10.0
...,...,...,...,...
194506,60000,2007-10-08,geog,9.0
194507,60000,2007-10-25,geog,14.0
194508,60000,2009-03-27,art,10.0
194509,60000,2010-01-07,child,14.0


generate the required code below for `first`, `last`, `book`, and `purch`,
and add the purchase frequencies for the different book types
hint: you can use pandas "value_counts" method here
hint: check the help for pandas' `first` and `last` methods

In [ ]:
## book means total amount spent on books, so to get that we do group by on acctnum and sum price

book_try = purchase_data.groupby(['acctnum'])['price'].sum()
book_try

acctnum
10001    109.0
10002     35.0
10003     25.0
10004     15.0
10005     15.0
         ...  
59996     15.0
59997     79.0
59998     15.0
59999     98.0
60000     60.0
Name: price, Length: 50000, dtype: float64

In [ ]:
#### From here we get the book variable
book_try_df = pd.DataFrame(book_try)
book_try_df = book_try_df.rename(columns= {'price' : 'book'})
book_try_df

,book
acctnum,
10001,109.0
10002,35.0
10003,25.0
10004,15.0
10005,15.0
...,...
59996,15.0
59997,79.0
59998,15.0


In [ ]:
book_try_df = book_try_df.reset_index()
book_try_df['acctnum'] = book_try_df['acctnum'].astype(int)

In [ ]:
book_try_df

,acctnum,book
0,10001,109.0
1,10002,35.0
2,10003,25.0
3,10004,15.0
4,10005,15.0
...,...,...
49995,59996,15.0
49996,59997,79.0
49997,59998,15.0
49998,59999,98.0


In [ ]:
## To get the purch variable i.e. number of books a user purchased (I'm thinking of using crosstabs)

purch_try = pd.crosstab(purchase_data.acctnum,purchase_data.purchase, margins = True, margins_name="purch")
purch_try

purchase,art,child,cook,do_it,geog,reference,youth,purch
acctnum,,,,,,,,
10001,0,3,2,0,2,1,2,10
10002,0,0,0,1,1,0,1,3
10003,0,0,2,0,0,0,0,2
10004,0,0,0,0,0,1,0,1
10005,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...
59997,1,3,1,1,1,0,0,7
59998,0,0,0,0,0,1,0,1
59999,1,2,1,0,2,2,0,8


In [ ]:
purch_try = purch_try.drop(['purch'], axis = 0)
purch_try

purchase,art,child,cook,do_it,geog,reference,youth,purch
acctnum,,,,,,,,
10001,0,3,2,0,2,1,2,10
10002,0,0,0,1,1,0,1,3
10003,0,0,2,0,0,0,0,2
10004,0,0,0,0,0,1,0,1
10005,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...
59996,0,1,0,0,0,0,0,1
59997,1,3,1,1,1,0,0,7
59998,0,0,0,0,0,1,0,1


In [ ]:
## Reordering the columns to match the needed output
purch_try = purch_try[['purch', 'child', 'youth', 'cook', 'do_it', 'reference', 'art', 'geog']]

In [ ]:
purch_try

purchase,purch,child,youth,cook,do_it,reference,art,geog
acctnum,,,,,,,,
10001,10,3,2,2,0,1,0,2
10002,3,0,1,0,1,0,0,1
10003,2,0,0,2,0,0,0,0
10004,1,0,0,0,0,1,0,0
10005,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
59996,1,1,0,0,0,0,0,0
59997,7,3,0,1,1,0,1,1
59998,1,0,0,0,0,1,0,0


In [ ]:
purch_try = purch_try.reset_index()

In [ ]:
purch_try['acctnum'] = purch_try['acctnum'].astype(int)

you may find the discussion below of interest at this point
https://stackoverflow.com/questions/65067042/pandas-frequency-of-a-specific-value-per-group

In [ ]:
## Merging begins
## M1 has Dem_data and try3
m1 = pd.merge(Dem_data,try3, on = ['acctnum','acctnum'])

In [ ]:
m1

,acctnum,gender,state,zip,zip3,first,last
0,10001,M,NY,10605,106,49,29
1,10002,M,NY,10960,109,39,27
2,10003,F,PA,19146,191,19,15
3,10004,F,NJ,07016,070,7,7
4,10005,F,NY,10804,108,15,15
...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9
49996,59997,F,NJ,08882,088,25,5
49997,59998,M,NJ,07410,074,3,3
49998,59999,M,NJ,07090,070,49,29


In [ ]:
print(m1.dtypes)
print(book_try_df.dtypes)

acctnum     int64
gender     object
state      object
zip        object
zip3       object
first       int64
last        int64
dtype: object
acctnum      int64
book       float64
dtype: object


In [ ]:
## Merging m1 and book_try_df

m2 = pd.merge(m1,book_try_df)

In [ ]:
m2

,acctnum,gender,state,zip,zip3,first,last,book
0,10001,M,NY,10605,106,49,29,109.0
1,10002,M,NY,10960,109,39,27,35.0
2,10003,F,PA,19146,191,19,15,25.0
3,10004,F,NJ,07016,070,7,7,15.0
4,10005,F,NY,10804,108,15,15,15.0
...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15.0
49996,59997,F,NJ,08882,088,25,5,79.0
49997,59998,M,NJ,07410,074,3,3,15.0
49998,59999,M,NJ,07090,070,49,29,98.0


In [ ]:
## Merging m2 and nonbook_data
m3 = pd.merge(m2,nonbook_data)

In [ ]:
m3

,acctnum,gender,state,zip,zip3,first,last,book,nonbook
0,10001,M,NY,10605,106,49,29,109.0,248
1,10002,M,NY,10960,109,39,27,35.0,103
2,10003,F,PA,19146,191,19,15,25.0,147
3,10004,F,NJ,07016,070,7,7,15.0,257
4,10005,F,NY,10804,108,15,15,15.0,134
...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15.0,12
49996,59997,F,NJ,08882,088,25,5,79.0,294
49997,59998,M,NJ,07410,074,3,3,15.0,178
49998,59999,M,NJ,07090,070,49,29,98.0,246


In [ ]:
m3['book'] = m3['book'].astype(int)

In [ ]:
m3['total'] = m3['book'] + m3['nonbook']

In [ ]:
m3

,acctnum,gender,state,zip,zip3,first,last,book,nonbook,total
0,10001,M,NY,10605,106,49,29,109,248,357
1,10002,M,NY,10960,109,39,27,35,103,138
2,10003,F,PA,19146,191,19,15,25,147,172
3,10004,F,NJ,07016,070,7,7,15,257,272
4,10005,F,NY,10804,108,15,15,15,134,149
...,...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15,12,27
49996,59997,F,NJ,08882,088,25,5,79,294,373
49997,59998,M,NJ,07410,074,3,3,15,178,193
49998,59999,M,NJ,07090,070,49,29,98,246,344


combine the different data frames using pandas' "merge" method

In [ ]:
## Merging m3 and purch_try

m4 = pd.merge(m3,purch_try)

In [ ]:
m4

,acctnum,gender,state,zip,zip3,first,last,book,nonbook,total,purch,child,youth,cook,do_it,reference,art,geog
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0
3,10004,F,NJ,07016,070,7,7,15,257,272,1,0,0,0,0,1,0,0
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15,12,27,1,1,0,0,0,0,0,0
49996,59997,F,NJ,08882,088,25,5,79,294,373,7,3,0,1,1,0,1,1
49997,59998,M,NJ,07410,074,3,3,15,178,193,1,0,0,0,0,1,0,0
49998,59999,M,NJ,07090,070,49,29,98,246,344,8,2,0,1,0,2,1,2


In [ ]:
## Final merge of m4 and buyer_data

bbb_rec = pd.merge(m4,buyer_data)

In [ ]:
bbb_rec

,acctnum,gender,state,zip,zip3,first,last,book,nonbook,total,purch,child,youth,cook,do_it,reference,art,geog,buyer,training
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2,no,1
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1,no,1
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0,no,0
3,10004,F,NJ,07016,070,7,7,15,257,272,1,0,0,0,0,1,0,0,no,0
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0,no,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15,12,27,1,1,0,0,0,0,0,0,no,1
49996,59997,F,NJ,08882,088,25,5,79,294,373,7,3,0,1,1,0,1,1,no,1
49997,59998,M,NJ,07410,074,3,3,15,178,193,1,0,0,0,0,1,0,0,no,1
49998,59999,M,NJ,07090,070,49,29,98,246,344,8,2,0,1,0,2,1,2,no,1


In [ ]:
bbb

,acctnum,gender,state,zip,zip3,first,last,book,nonbook,total,purch,child,youth,cook,do_it,reference,art,geog,buyer,training
0,10001,M,NY,10605,106,49,29,109,248,357,10,3,2,2,0,1,0,2,no,1
1,10002,M,NY,10960,109,39,27,35,103,138,3,0,1,0,1,0,0,1,no,1
2,10003,F,PA,19146,191,19,15,25,147,172,2,0,0,2,0,0,0,0,no,0
3,10004,F,NJ,07016,070,7,7,15,257,272,1,0,0,0,0,1,0,0,no,0
4,10005,F,NY,10804,108,15,15,15,134,149,1,0,0,1,0,0,0,0,no,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,59996,F,NY,11967,119,9,9,15,12,27,1,1,0,0,0,0,0,0,no,1
49996,59997,F,NJ,08882,088,25,5,79,294,373,7,3,0,1,1,0,1,1,no,1
49997,59998,M,NJ,07410,074,3,3,15,178,193,1,0,0,0,0,1,0,0,no,1
49998,59999,M,NJ,07090,070,49,29,98,246,344,8,2,0,1,0,2,1,2,no,1


In [ ]:
bbb['acctnum'] = bbb['acctnum'].astype("int32")
bbb_rec['acctnum'] = bbb_rec['acctnum'].astype("int32")
bbb['acctnum'] = bbb['acctnum'].astype("object")
bbb_rec['acctnum'] = bbb_rec['acctnum'].astype("object")
bbb_rec['gender'] = bbb_rec['gender'].astype("category")
bbb_rec['state'] = bbb_rec['state'].astype("category")
bbb_rec['first'] = bbb_rec['first'].astype("int32")
bbb_rec['last'] = bbb_rec['last'].astype("int32")
bbb_rec['book'] = bbb_rec['book'].astype("int32")
bbb_rec['nonbook'] = bbb_rec['nonbook'].astype("int32")
bbb_rec['total'] = bbb_rec['total'].astype("int32")
bbb_rec['purch'] = bbb_rec['purch'].astype("int32")
bbb_rec['child'] = bbb_rec['child'].astype("int32")
bbb_rec['youth'] = bbb_rec['youth'].astype("int32")
bbb_rec['cook'] = bbb_rec['cook'].astype("int32")
bbb_rec['do_it'] = bbb_rec['do_it'].astype("int32")
bbb_rec['reference'] = bbb_rec['reference'].astype("int32")
bbb_rec['art'] = bbb_rec['art'].astype("int32")
bbb_rec['geog'] = bbb_rec['geog'].astype("int32")
bbb_rec['buyer'] = bbb_rec['buyer'].astype("category")
bbb_rec['training'] = bbb_rec['training'].astype("int32")

In [ ]:
# bbb['acctnum'] = bbb['acctnum'].astype("object")
# bbb_rec['acctnum'] = bbb_rec['acctnum'].astype("object")

check if the columns in bbb and bbb_rec are in the same order
and are of the same type - fix as needed

In [ ]:
pd.DataFrame(
    {
        "bbb_names": bbb.columns,
        "bbb_types": bbb.dtypes.astype(str).values,
        "bbb_rec_names": bbb_rec.columns,
        "bbb_rec_types": bbb_rec.dtypes.astype(str),
        "check_names": bbb.columns == bbb_rec.columns,
        "check_types": bbb.dtypes.astype(str).values == bbb_rec.dtypes.astype(str).values
    }
)

,bbb_names,bbb_types,bbb_rec_names,bbb_rec_types,check_names,check_types
acctnum,acctnum,object,acctnum,object,True,True
gender,gender,category,gender,category,True,True
state,state,category,state,category,True,True
zip,zip,object,zip,object,True,True
zip3,zip3,object,zip3,object,True,True
first,first,int32,first,int32,True,True
last,last,int32,last,int32,True,True
book,book,int32,book,int32,True,True
nonbook,nonbook,int32,nonbook,int32,True,True
total,total,int32,total,int32,True,True


In [ ]:
##########

In [ ]:
## Adding the description as metadata
description = "# BookBinders Book Club\n\nInformation on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below\n\n## Variables\n\n* acctnum: Customer account number\n* gender: Customer gender - M=male, F=female\n* state: State where customer lives (2-character abbreviation)\n* zip: ZIP code (5-digit)\n* zip3: First 3 digits of ZIP code\n* first: Number of months since first purchase\n* last: Number of months since most recent purchase\n* book: Total dollars spent on books\n* nonbook: Total dollars spent on non-book products\n* total: Total dollars spent\n* purch: Total number of books purchased\n* child: Total number of children's books purchased\n* youth: Total number of youth books purchased\n* cook: Total number of cook books purchased \n* do_it: Total number of do-it-yourself books purchased \n* reference: Total number of reference books purchased \n* art: Total number of art books purchased \n* geog: Total number of geography books purchased \n* buyer: Did the customer buy The Art History of Florence?  (yes, no)\n* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset"
p = "data/bbb_description.txt"
bbb_rec = rsm.add_description(bbb_rec, md=description, path =p)

add the description as metadata to bbb_rec (see data/bbb_description.txt)
see https://stackoverflow.com/a/40514650/1974918 for more information

In [ ]:
x = bbb_rec['acctnum']
y = bbb['acctnum']
data = {
    "Acc_BBB_REC" : x,
    "Acc_BBB" : y
}

In [ ]:
qwerty = pd.concat(data, axis = 1)

In [ ]:
comparison_column = np.where(qwerty["Acc_BBB_REC"] == qwerty["Acc_BBB"], True, False)
qwerty["equal"] = comparison_column

In [ ]:
qwerty

,Acc_BBB_REC,Acc_BBB,equal
0,10001,10001,True
1,10002,10002,True
2,10003,10003,True
3,10004,10004,True
4,10005,10005,True
...,...,...,...
49995,59996,59996,True
49996,59997,59997,True
49997,59998,59998,True
49998,59999,59999,True


In [ ]:
qwerty.dtypes

Acc_BBB_REC    object
Acc_BBB        object
equal            bool
dtype: object

In [ ]:
print((bbb_rec["acctnum"] == bbb["acctnum"]).sum())
print((bbb_rec["gender"] == bbb["gender"]).sum())
print((bbb_rec["state"] == bbb["state"]).sum())
print((bbb_rec["zip"] == bbb["zip"]).sum())
print((bbb_rec["zip3"] == bbb["zip3"]).sum())
print((bbb_rec["first"] == bbb["first"]).sum())
print((bbb_rec["last"] == bbb["last"]).sum())
print((bbb_rec["book"] == bbb["book"]).sum())
print((bbb_rec["nonbook"] == bbb["nonbook"]).sum())
print((bbb_rec["total"] == bbb["total"]).sum())
print((bbb_rec["purch"] == bbb["purch"]).sum())
print((bbb_rec["child"] == bbb["child"]).sum())
print((bbb_rec["youth"] == bbb["youth"]).sum())
print((bbb_rec["cook"] == bbb["cook"]).sum())
print((bbb_rec["do_it"] == bbb["do_it"]).sum())
print((bbb_rec["reference"] == bbb["reference"]).sum())
print((bbb_rec["art"] == bbb["art"]).sum())
print((bbb_rec["geog"] == bbb["geog"]).sum())
print((bbb_rec["buyer"] == bbb["buyer"]).sum())
print((bbb_rec["training"] == bbb["training"]).sum())

50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000
50000


In [ ]:
bbb_rec['acctnum'].isin(bbb['acctnum']).value_counts()

True    50000
Name: acctnum, dtype: int64

check that you get the same output for both bbb and bbb_rec

In [ ]:
rsm.describe(bbb_rec)
rsm.describe(bbb)

# BookBinders Book Club

Information on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below

## Variables

* acctnum: Customer account number
* gender: Customer gender - M=male, F=female
* state: State where customer lives (2-character abbreviation)
* zip: ZIP code (5-digit)
* zip3: First 3 digits of ZIP code
* first: Number of months since first purchase
* last: Number of months since most recent purchase
* book: Total dollars spent on books
* nonbook: Total dollars spent on non-book products
* total: Total dollars spent
* purch: Total number of books purchased
* child: Total number of children's books purchased
* youth: Total number of youth books purchased
* cook: Total number of cook books purchased 
* do_it: Total number of do-it-yourself books purchased 
* reference: Total number of reference books purchased 
* art: Total number of art books purchased 
* geog: Total number of geography books purchased 
* buyer: Did the customer buy The Art History of Florence?  (yes, no)
* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset

# BookBinders Book Club

Information on purchasing history and demographics for 50,000 of the BookBinders Book Club's customers'. Variables included in the dataset are described below

## Variables

* acctnum: Customer account number
* gender: Customer gender - M=male, F=female
* state: State where customer lives (2-character abbreviation)
* zip: ZIP code (5-digit)
* zip3: First 3 digits of ZIP code
* first: Number of months since first purchase
* last: Number of months since most recent purchase
* book: Total dollars spent on books
* nonbook: Total dollars spent on non-book products
* total: Total dollars spent
* purch: Total number of books purchased
* child: Total number of children's books purchased
* youth: Total number of youth books purchased
* cook: Total number of cook books purchased 
* do_it: Total number of do-it-yourself books purchased 
* reference: Total number of reference books purchased 
* art: Total number of art books purchased 
* geog: Total number of geography books purchased 
* buyer: Did the customer buy The Art History of Florence?  (yes, no)
* training: Dummy variable that splits the dataset into a training (1) and validation (0) dataset

# DO NOT EDIT CODE BELOW THIS CELL
# YOUR CODE MUST PASS THE TEST BELOW

In [ ]:
test1 = bbb_rec.equals(bbb)
if hasattr(bbb_rec, "description"):
    test2 = bbb_rec.description == bbb.description
else:
    test2 = False

if test1 is True and test2 is True:
    print("Well done! Both tests passed!")
    print("bbb_rec will now be written to the data directory")
    bbb_rec.to_pickle("data/bbb_rec.pkl")
else:
    test = False
    if test1 is False:
        raise Exception(
            """Test of equality of data frames failed.
            Use bbb.dtypes and bbb_rec.dtypes to check
            for differences in types. Check the number
            of mistakes per colmun using, for example,
            (bbb_rec["book"] == bbb["book"]).sum()"""
        )
    if test2 is False:
        raise Exception(
            """Add a description to the bbb_rec data frame.
            Read the description from the txt file in the
            data directory. See
            https://stackoverflow.com/a/40514650/1974918
            for more information"""
        )

Well done! Both tests passed!
bbb_rec will now be written to the data directory


In [ ]:
################################FIN######################################